##Dasar Teori

**Validasi klasifikasi**

Metode validasi digunakan untuk menguji sejauh mana akurasi klasifikasi dari
metode klasifikasi yang dikembangkan. Metode ini dikenal dengan cross-validation dimana
data masukan dipecah menjadi data latih dan data uji. Pada metode ini terdapat beberapa
metode, metode yang paling sederhana adalah adalah 2-fold cross-validation dimana data
dipisah secara acak menjadi dalam latih dan data uji. Pengembangan dari 2-fold
cross-validation adalah K-fold cross-validation dimana data dipisah secara acak ke dalam K
subset. Untuk pelatihan subset K-1 dipilih dan sisanya digunakan untuk pengujian. Proses
ini diulangi sampai semua subset digunakan untuk pengujian. Versi lain dari K-fold
cross-validation adalah Leave-one out cross-validation (LOOCV) diman K sama dengan
jumlah sampel yakni masing-masing sampel digunakan untuk testing dan sisanya digunakan
untuk pelatihan. Proses ini diulangi sampai semua sampel diuji.

Tahap selanjutnya data pelatihan akan digunakan untuk membagun model dengan menentukan parameter terbaik dari masing-masing model berdasrakan dataset yang ada. Kinerja terbaik dari suatu model yang dikembangkan diukur menggunakan beberap tolok ukur sebagai berikut :
- presison
- recall
- specifity
- akurasi
Masing-masing metrik tersebut ditentukan berdasarkan matrik kebingungan.



**Confusion matrik**

Matriks kebingungan atau matriks kesalahan adalah suatu tabel yang merepresentasikan kinerja suatu algoritma. Setiap baris matriks mewakili instance di kelas yang diprediksi sementara setiap kolom mewakili instance di kelas yang sebenarnya (atau sebaliknya). 

#Beberapa tahapan dalam valisasi klasifikasi


Beberapa metode klasifikasi yang telah dipelajari mempunyai kelebihan dan kekurannya. Perbandingan kinerja antar metode klasifikasi ini diukur menggunakan beberapa metrik.  Pada praktikum ini akan dibahas beberapa metrik untuk mengukur kinerja klasifikasi. 
Tahapan untuk mengevaluasi suatu metode klasifikasi dilakukan sebagai berikut :
- Pisahkan set data validasi.
- Siapkan test harness untuk menggunakan validasi silang 10 kali lipat.
- Buat beberapa model berbeda untuk memprediksi spesies dari pengukuran bunga
- Pilih model terbaik.

##1. Tentukan metode klasifikasi yang akan dievaluasi

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [2]:
# Spot Check Algorithms
models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

# evaluate each model in turn
results = []
names = []

##2. Baca dataset
Pada contoh ini terdapat tiga buah dataset yakni :
- dataR2
- iris data 
- breast cancer

Kita akan gunakan dataR2 sebagai contoh sedangkan dua data berikunya akan digunakan sabagai latihan.

In [3]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv
# ! wget https://dataset-ppm.s3.amazonaws.com/iris.csv
# ! wget https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv

--2021-05-04 08:02:30--  https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7665 (7.5K) [application/x-httpd-php]
Saving to: ‘dataR2.csv.24’

dataR2.csv.24       100%[===================>]   7.49K  --.-KB/s    in 0s      

2021-05-04 08:02:30 (130 MB/s) - ‘dataR2.csv.24’ saved [7665/7665]



In [4]:
import pandas as pd
import numpy as np
# dataset = pd.read_csv('iris.csv')
# dataset = pd.read_csv('breast_cancer.csv')
dataset = pd.read_csv('dataR2.csv')
dataset.head()

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1


##3.1 Membuat Set Data Validasi
Tujuan evaluasi adalah memperkirakan keakuratan model terbaik pada data tak terlihat dengan mengevaluasinya pada data tak terlihat aktual. Evaluasi ini dilakukan dengan cara membagi dataset menjadi dua bagian yakni sebagian digunakan sebagai data latih dan sebagian yang lain digunakan sebagai data uji. Beberapa praktikum sebelumnya telah dipraktekkan dengan menggunakan metode train_test_split : 
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)


Sebelum membuat data latih dan data uji. Langkah pertama pisahkan antara data attribute dan data label suatu dataset. Karena datasaet data2R terdiri dari 9 attribut dan 1 label.  Maka kita catat X sebagai data dan Y sebagai label.

In [5]:
from sklearn.model_selection import train_test_split

array = dataset.values
X = array[:,0:9]
y = array[:,9]

X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)

##3.2 K-Fold Cross Validation
k-Fold Cross-Validation Validasi silang adalah prosedur pengambilan sampel ulang yang digunakan untuk mengevaluasi model pembelajaran mesin pada sampel data terbatas. Prosedur ini memiliki parameter tunggal yang disebut k yang mengacu pada jumlah grup yang akan dipecah menjadi sampel data tertentu. Karena itu, prosedur ini sering disebut k-fold cross-validation. Ketika nilai spesifik untuk k dipilih, nilai tersebut dapat digunakan sebagai pengganti k dalam referensi ke model, seperti k = 10 menjadi validasi silang 10 kali lipat.

Validasi silang terutama digunakan dalam pembelajaran mesin terapan untuk memperkirakan kemampuan model pembelajaran mesin pada data yang tidak terlihat. Yaitu, menggunakan sampel terbatas untuk memperkirakan bagaimana performa model secara umum saat digunakan untuk membuat prediksi pada data yang tidak digunakan selama pelatihan model.

Prosedur umumnya adalah sebagai berikut:
- Acak kumpulan data berdasarkan id
- Pisahkan kumpulan data menjadi kelompok k
- Untuk setiap grup unik:
  - Ambil kelompok sebagai kelompok data uji
  - Ambil kelompok yang tersisa sebagai kumpulan data pelatihan
  - Catat hasil pengujian masing-masing group

K-Fold ini digunakan untuk validasi model yang digunakan. Evaluasi dilakukan pada dataset latih. Pada praktikum ini digunakan k=10, berarti akan membagi dataset latih menjadi 10 bagian. Satu bagian digunakan untuk testing dan sisanya sebagai data latih.

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
 
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

KNN: 0.487778 (0.143540)
NB: 0.662222 (0.125629)
SVM: 0.465556 (0.047778)


##4. Buat Prediksi

Kita harus memilih algoritma yang akan digunakan untuk membuat prediksi.

##4.1 Membuat Prediksi
Kita dapat menyesuaikan model pada seluruh dataset pelatihan dan membuat prediksi pada dataset validasi.

In [7]:
model = SVC(gamma='auto')

model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

# 4.2 Evaluasi Prediksi
Kami dapat mengevaluasi prediksi dengan membandingkannya dengan hasil yang diharapkan dalam set validasi, lalu menghitung akurasi klasifikasi, serta matriks kebingungan dan laporan klasifikasi.

In [8]:
# create dataset
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [9]:
from sklearn.metrics import classification_report

print(classification_report(Y_validation, predictions))

              precision    recall  f1-score   support

         1.0       0.25      1.00      0.40         6
         2.0       0.00      0.00      0.00        18

    accuracy                           0.25        24
   macro avg       0.12      0.50      0.20        24
weighted avg       0.06      0.25      0.10        24



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Tugas :

1. Lakukan perbandingan klasifikasi antara SVM, Naive Bayes, KNN untuk dataset iris dan breast cancer
2. Pada kedua dataset lakukan proses normaslisai data dan preproses data untuk menengani data yang hilang jika ada.

# **Nomor 1 dataset iris**

Lakukan perbandingan klasifikasi antara SVM, Naive Bayes, KNN untuk dataset iris dan breast cancer

In [10]:
! wget https://dataset-ppm.s3.amazonaws.com/iris.csv

--2021-05-04 08:02:31--  https://dataset-ppm.s3.amazonaws.com/iris.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.216.97.155
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.216.97.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4609 (4.5K) [text/csv]
Saving to: ‘iris.csv.24’

iris.csv.24         100%[===================>]   4.50K  --.-KB/s    in 0s      

2021-05-04 08:02:31 (126 MB/s) - ‘iris.csv.24’ saved [4609/4609]



In [11]:
dataset_iris = pd.read_csv('iris.csv')

In [12]:
dataset_iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [13]:
from sklearn.model_selection import train_test_split

array_iris = dataset_iris.values
X_iris = array_iris[:,0:4]
y_iris = array_iris[:,4]

X_iris_train, X_iris_validation, Y_iris_train, Y_iris_validation = train_test_split(X_iris, y_iris, test_size=0.20, random_state=1, shuffle=True)

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
 
	cv_results_iris = cross_val_score(model, X_iris_train, Y_iris_train, cv=kfold, scoring='accuracy')
	results.append(cv_results_iris)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results_iris.mean(), cv_results_iris.std()))

KNN: 0.958333 (0.041667)
NB: 0.950000 (0.055277)
SVM: 0.983333 (0.033333)


In [15]:
#Menggunakan KNN, dengan k = 9
model_iris_KNN = KNeighborsClassifier(n_neighbors=9)

model_iris_KNN.fit(X_iris_train, Y_iris_train)
predictions_iris_KNN = model_iris_KNN.predict(X_iris_validation)

In [16]:
# create dataset untuk KNN
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [17]:
from sklearn.metrics import classification_report

print(classification_report(Y_iris_validation, predictions_iris_KNN))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       1.00      0.92      0.96        13
 Iris-virginica       0.86      1.00      0.92         6

       accuracy                           0.97        30
      macro avg       0.95      0.97      0.96        30
   weighted avg       0.97      0.97      0.97        30



In [18]:
#Menggunakan Naive Bayes Gaussian

In [19]:
model_iris_NB = GaussianNB()

model_iris_NB.fit(X_iris_train, Y_iris_train)
predictions_iris_NB = model_iris_NB.predict(X_iris_validation)

In [20]:
# create dataset untuk NB
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [21]:
from sklearn.metrics import classification_report

print(classification_report(Y_iris_validation, predictions_iris_NB))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       1.00      0.92      0.96        13
 Iris-virginica       0.86      1.00      0.92         6

       accuracy                           0.97        30
      macro avg       0.95      0.97      0.96        30
   weighted avg       0.97      0.97      0.97        30



In [22]:
#Menggunakan Support Vector Machine
model_iris_SVM = SVC(gamma='auto')

model_iris_SVM.fit(X_iris_train, Y_iris_train)
predictions_iris_SVM = model_iris_SVM.predict(X_iris_validation)

In [23]:
# create dataset untuk SVM
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [24]:
from sklearn.metrics import classification_report

print(classification_report(Y_iris_validation, predictions_iris_SVM))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       1.00      0.92      0.96        13
 Iris-virginica       0.86      1.00      0.92         6

       accuracy                           0.97        30
      macro avg       0.95      0.97      0.96        30
   weighted avg       0.97      0.97      0.97        30



# **Nomor 1 dataset breast cancer**

In [25]:
! wget https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv

--2021-05-04 08:02:32--  https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3664 (3.6K) [text/plain]
Saving to: ‘breast_cancer.csv.24’

breast_cancer.csv.2 100%[===================>]   3.58K  --.-KB/s    in 0s      

2021-05-04 08:02:32 (53.9 MB/s) - ‘breast_cancer.csv.24’ saved [3664/3664]



In [26]:
dataset_cancer = pd.read_csv('breast_cancer.csv')
dataset_cancer = dataset_cancer.fillna(0)
dataset_cancer

,id,clump_thickness,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nucklei,bland_chromatin,normal_nucleoli,mitoses,diagnosis
0,1000025,5.0,1.0,0.0,1.0,0.0,1.0,3.0,1.0,1.0,benign
1,1002945,5.0,4.0,4.0,5.0,7.0,0.0,3.0,0.0,1.0,benign
2,1015425,3.0,1.0,1.0,1.0,0.0,2.0,3.0,1.0,1.0,benign
3,1016277,6.0,8.0,8.0,1.0,3.0,0.0,3.0,7.0,1.0,benign
4,1017023,4.0,1.0,1.0,3.0,0.0,1.0,3.0,1.0,0.0,benign
...,...,...,...,...,...,...,...,...,...,...,...
95,1167439,2.0,3.0,4.0,4.0,2.0,5.0,2.0,5.0,1.0,malignant
96,1168359,8.0,2.0,3.0,0.0,6.0,3.0,7.0,1.0,1.0,malignant
97,1168736,10.0,10.0,10.0,10.0,10.0,1.0,8.0,8.0,8.0,malignant
98,1169049,7.0,3.0,4.0,4.0,3.0,3.0,3.0,2.0,7.0,malignant


In [27]:
from sklearn.model_selection import train_test_split

array_cancer = dataset_cancer.values
X_cancer = array_cancer[:,1:10]
y_cancer = array_cancer[:,10]

X_cancer_train, X_cancer_validation, Y_cancer_train, Y_cancer_validation = train_test_split(X_cancer, y_cancer, test_size=0.20, random_state=1, shuffle=True)

In [28]:
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
 
	cv_results_cancer = cross_val_score(model, X_cancer_train, Y_cancer_train, cv=kfold, scoring='accuracy')
	results.append(cv_results_cancer)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results_cancer.mean(), cv_results_cancer.std()))

KNN: 0.925000 (0.100000)
NB: 0.912500 (0.097628)
SVM: 0.912500 (0.080039)


In [29]:
#Menggunakan KNN, dengan k = 13
model_cancer_KNN = KNeighborsClassifier(n_neighbors=13)

model_cancer_KNN.fit(X_cancer_train, Y_cancer_train)
predictions_cancer_KNN = model_cancer_KNN.predict(X_cancer_validation)

In [30]:
# create dataset KNN
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [31]:
from sklearn.metrics import classification_report

print(classification_report(Y_cancer_validation, predictions_cancer_KNN))

              precision    recall  f1-score   support

      benign       0.90      1.00      0.95         9
   malignant       1.00      0.91      0.95        11

    accuracy                           0.95        20
   macro avg       0.95      0.95      0.95        20
weighted avg       0.96      0.95      0.95        20



In [32]:
#Menggunakan Naive Bayes

In [33]:
model_cancer_NB = GaussianNB()

model_cancer_NB.fit(X_cancer_train, Y_cancer_train)
predictions_cancer_NB = model_cancer_NB.predict(X_cancer_validation)

In [34]:
# create dataset NB
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [35]:
from sklearn.metrics import classification_report

print(classification_report(Y_cancer_validation, predictions_cancer_NB))

              precision    recall  f1-score   support

      benign       0.90      1.00      0.95         9
   malignant       1.00      0.91      0.95        11

    accuracy                           0.95        20
   macro avg       0.95      0.95      0.95        20
weighted avg       0.96      0.95      0.95        20



In [36]:
#Menggunakan Support Vector Machine
model_cancer_SVM = SVC(gamma='auto')

model_cancer_SVM.fit(X_cancer_train, Y_cancer_train)
predictions_cancer_SVM = model_cancer_SVM.predict(X_cancer_validation)

In [37]:
# create dataset SVM
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [38]:
from sklearn.metrics import classification_report

print(classification_report(Y_cancer_validation, predictions_cancer_SVM))

              precision    recall  f1-score   support

      benign       1.00      0.89      0.94         9
   malignant       0.92      1.00      0.96        11

    accuracy                           0.95        20
   macro avg       0.96      0.94      0.95        20
weighted avg       0.95      0.95      0.95        20



# **Nomor 2 dataset iris**

Pada kedua dataset lakukan proses normaslisai data dan preproses data untuk menengani data yang hilang jika ada.

In [39]:
dataset_iris_test = pd.read_csv('iris.csv')
dataset_iris_test

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [40]:
#Memeriksa apakah ada data null
dataset_iris_test.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

Dapat dilihat tidak ada data yang bernilai Null

In [41]:
dataset_iris_test.duplicated().sum()

3

In [42]:
dataset_iris_test.drop_duplicates(keep="first")

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [43]:
from sklearn.preprocessing import MinMaxScaler

array_iris_test = dataset_iris_test.values
scaler = MinMaxScaler()
dataset_fitur = scaler.fit_transform(array_iris_test[:,:-1])

In [44]:
from sklearn.model_selection import train_test_split

array_iris_MM = dataset_iris_test.values
X_iris_MM = dataset_fitur
y_iris_MM = array_iris_MM[:,4]

X_iris_train_MM, X_iris_validation_MM, Y_iris_train_MM, Y_iris_validation_MM = train_test_split(X_iris_MM, y_iris_MM, test_size=0.20, random_state=1, shuffle=True)

In [45]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
 
	cv_results_iris = cross_val_score(model, X_iris_train_MM, Y_iris_train_MM, cv=kfold, scoring='accuracy')
	results.append(cv_results_iris)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results_iris.mean(), cv_results_iris.std()))

KNN: 0.950000 (0.040825)
NB: 0.950000 (0.055277)
SVM: 0.941667 (0.053359)


In [46]:
#Menggunakan KNN, dengan k = 13
model_iris_KNN_MM = KNeighborsClassifier(n_neighbors=13)

model_iris_KNN_MM.fit(X_iris_train_MM, Y_iris_train_MM)
predictions_iris_KNN_MM = model_iris_KNN_MM.predict(X_iris_validation_MM)

In [47]:
# create dataset untuk KNN
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [48]:
from sklearn.metrics import classification_report

print(classification_report(Y_iris_validation_MM, predictions_iris_KNN_MM))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       1.00      0.92      0.96        13
 Iris-virginica       0.86      1.00      0.92         6

       accuracy                           0.97        30
      macro avg       0.95      0.97      0.96        30
   weighted avg       0.97      0.97      0.97        30



In [49]:
#Menggunakan NAive Bayes
model_iris_NB_MM = GaussianNB()

model_iris_NB_MM.fit(X_iris_train_MM, Y_iris_train_MM)
predictions_iris_NB_MM = model_iris_NB_MM.predict(X_iris_validation_MM)

In [50]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [51]:
from sklearn.metrics import classification_report

print(classification_report(Y_iris_validation_MM, predictions_iris_NB_MM))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       1.00      0.92      0.96        13
 Iris-virginica       0.86      1.00      0.92         6

       accuracy                           0.97        30
      macro avg       0.95      0.97      0.96        30
   weighted avg       0.97      0.97      0.97        30



In [52]:
#Menggunakan SVM
model_iris_SVM_MM = SVC(gamma='auto')

model_iris_SVM_MM.fit(X_iris_train_MM, Y_iris_train_MM)
predictions_iris_SVM_MM = model_iris_SVM_MM.predict(X_iris_validation_MM)

In [53]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [54]:
from sklearn.metrics import classification_report

print(classification_report(Y_iris_validation_MM, predictions_iris_SVM_MM))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       1.00      0.92      0.96        13
 Iris-virginica       0.86      1.00      0.92         6

       accuracy                           0.97        30
      macro avg       0.95      0.97      0.96        30
   weighted avg       0.97      0.97      0.97        30



# **Nomor 2 dataset breast cancer**

In [55]:
dataset_cancer_test = pd.read_csv('breast_cancer.csv')
dataset_cancer_test

,id,clump_thickness,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nucklei,bland_chromatin,normal_nucleoli,mitoses,diagnosis
0,1000025,5.0,1.0,NaN,1.0,NaN,1.0,3.0,1.0,1.0,benign
1,1002945,5.0,4.0,4.0,5.0,7.0,NaN,3.0,NaN,1.0,benign
2,1015425,3.0,1.0,1.0,1.0,NaN,2.0,3.0,1.0,1.0,benign
3,1016277,6.0,8.0,8.0,1.0,3.0,NaN,3.0,7.0,1.0,benign
4,1017023,4.0,1.0,1.0,3.0,NaN,1.0,3.0,1.0,NaN,benign
...,...,...,...,...,...,...,...,...,...,...,...
95,1167439,2.0,3.0,4.0,4.0,2.0,5.0,2.0,5.0,1.0,malignant
96,1168359,8.0,2.0,3.0,NaN,6.0,3.0,7.0,1.0,1.0,malignant
97,1168736,10.0,10.0,10.0,10.0,10.0,1.0,8.0,8.0,8.0,malignant
98,1169049,7.0,3.0,4.0,4.0,3.0,3.0,3.0,2.0,7.0,malignant


In [56]:
dataset_cancer_test.isnull().sum()

id                       0
clump_thickness         10
cell_size               10
cell_shape              10
marginal_adhesion       10
epithelial_cell_size    10
bare_nucklei            10
bland_chromatin         10
normal_nucleoli         10
mitoses                 10
diagnosis                0
dtype: int64

In [57]:
#Karena adanya missing value maka perlu diperbaiki
#Disini saya menggunakan rata - rata untuk mengisinya

list_fitur = dataset_cancer_test.columns[:-1]
for column in list_fitur:
  dataset_cancer_test[column] = dataset_cancer_test[column].fillna(dataset_cancer_test[column].mean())

dataset_cancer_test.head()

,id,clump_thickness,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nucklei,bland_chromatin,normal_nucleoli,mitoses,diagnosis
0,1000025,5.0,1.0,3.7,1.0,3.733333,1.0,3.0,1.000000,1.000000,benign
1,1002945,5.0,4.0,4.0,5.0,7.000000,3.7,3.0,3.477778,1.000000,benign
2,1015425,3.0,1.0,1.0,1.0,3.733333,2.0,3.0,1.000000,1.000000,benign
3,1016277,6.0,8.0,8.0,1.0,3.000000,3.7,3.0,7.000000,1.000000,benign
4,1017023,4.0,1.0,1.0,3.0,3.733333,1.0,3.0,1.000000,2.133333,benign


In [58]:
from sklearn.preprocessing import MinMaxScaler

array_cancer_test = dataset_cancer_test.values
scaler = MinMaxScaler()
dataset_fitur = scaler.fit_transform(array_cancer_test[:,:-1])

In [59]:
from sklearn.model_selection import train_test_split

array_cancer_MM = dataset_cancer_test.values
X_cancer_MM = dataset_fitur
y_cancer_MM = array_cancer_MM[:,10]

X_cancer_train_MM, X_cancer_validation_MM, Y_cancer_train_MM, Y_cancer_validation_MM = train_test_split(X_cancer_MM, y_cancer_MM, test_size=0.20, random_state=1, shuffle=True)

In [60]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
 
	cv_results_cancer = cross_val_score(model, X_cancer_train_MM, Y_cancer_train_MM, cv=kfold, scoring='accuracy')
	results.append(cv_results_cancer)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results_cancer.mean(), cv_results_cancer.std()))

KNN: 0.925000 (0.100000)
NB: 0.912500 (0.097628)
SVM: 0.912500 (0.097628)


In [61]:
#Menggunakan KNN, dengan k = 13
model_cancer_KNN_MM = KNeighborsClassifier(n_neighbors=13)

model_cancer_KNN_MM.fit(X_cancer_train_MM, Y_cancer_train_MM)
predictions_cancer_KNN_MM = model_cancer_KNN_MM.predict(X_cancer_validation_MM)

In [62]:
# create dataset untuk KNN
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [63]:
from sklearn.metrics import classification_report

print(classification_report(Y_cancer_validation_MM, predictions_cancer_KNN_MM))

              precision    recall  f1-score   support

      benign       0.90      1.00      0.95         9
   malignant       1.00      0.91      0.95        11

    accuracy                           0.95        20
   macro avg       0.95      0.95      0.95        20
weighted avg       0.96      0.95      0.95        20



In [64]:
#Menggunakan Naive Bayes
model_cancer_NB_MM = GaussianNB()

model_cancer_NB_MM.fit(X_cancer_train_MM, Y_cancer_train_MM)
predictions_cancer_NB_MM = model_cancer_NB_MM.predict(X_cancer_validation_MM)

In [65]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [66]:
from sklearn.metrics import classification_report

print(classification_report(Y_cancer_validation_MM, predictions_cancer_NB_MM))

              precision    recall  f1-score   support

      benign       0.90      1.00      0.95         9
   malignant       1.00      0.91      0.95        11

    accuracy                           0.95        20
   macro avg       0.95      0.95      0.95        20
weighted avg       0.96      0.95      0.95        20



In [67]:
#Menggunakan Support Vector Machine
model_cancer_SVM_MM = SVC(gamma='auto')

model_cancer_SVM_MM.fit(X_cancer_train_MM, Y_cancer_train_MM)
predictions_cancer_SVM_MM = model_cancer_SVM_MM.predict(X_cancer_validation_MM)

In [68]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)

In [69]:
from sklearn.metrics import classification_report

print(classification_report(Y_cancer_validation_MM, predictions_cancer_SVM_MM))

              precision    recall  f1-score   support

      benign       0.90      1.00      0.95         9
   malignant       1.00      0.91      0.95        11

    accuracy                           0.95        20
   macro avg       0.95      0.95      0.95        20
weighted avg       0.96      0.95      0.95        20

